In [88]:
import urllib
from urllib.request import urlopen
import numpy as np
import pandas as pd
import lxml
from lxml import html
from bs4 import BeautifulSoup
from bs4 import *
import xml.etree.cElementTree as ET
import cerberus
import re
import sqlite3
import os
import html5lib
import json
import sys
import requests
import time
import random
from collections import defaultdict

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [59]:
browser.close()

In [6]:
songs_lyrics = pd.read_csv('songs_scaled.csv')

In [71]:
songs['lyrics'] = pd.Series()

In [72]:
missing_songs = defaultdict(list)

def get_lyrics(songs):
    for index, row in songs.iterrows():
        artist = songs.loc[index, 'artist']
        track = songs.loc[index, 'track']
        
        browser = webdriver.Chrome()
        browser.get('http://www.google.com')

        search = browser.find_element_by_name('q')
        search.send_keys("{} {} lyrics freak".format(artist, track)) #grab lyrics from lyricsfreak.com
        search.send_keys(Keys.RETURN) # hit return after you enter search text
        try:
            link = browser.find_element_by_partial_link_text('Lyrics Freak')
            url = (link.get_attribute('href'))
            soup = BeautifulSoup(urlopen(url), 'lxml')
            lyric = re.sub('<br/>', ' ', str(soup.findAll('div', {'id': 'content_h'})[0]))
            lyrics = re.findall(r'<div class="dn" id="content_h">(.*)</div>', lyric)[0]
            songs.loc[index, 'lyrics'] = lyrics                      
        except:
            missing_songs[index] = (artist, track)
            sys.stdout.write('\r skipped: {}'.format(missing_songs)
            pass
        browser.close()
        

In [409]:
get_lyrics(songs)

In [129]:
still_missing = defaultdict(list)

def get_rejects(missing_songs):
    global songs
    for index in missing_songs:
        artist = songs.loc[index, 'artist']
        track = songs.loc[index, 'track']
        
        ###getting rid of trash
        try:
            artist = re.findall(r'(.*) Featuring',artist)[0]
        except:
            pass
        
        try:
            track = re.findall(r'(.*) \(.*\)',track)[0]
        except:
            pass
        
        browser = webdriver.Chrome()
        browser.get('http://www.google.com')

        search = browser.find_element_by_name('q')
        search.send_keys("{} {} lyrics freak".format(artist, track))
        search.send_keys(Keys.RETURN) # hit return after you enter search text
        try:
            link = browser.find_element_by_partial_link_text('Lyrics Freak')
            url = (link.get_attribute('href'))
            soup = BeautifulSoup(urlopen(url), 'lxml')
            lyric = re.sub('<br/>', ' ', str(soup.findAll('div', {'id': 'content_h'})[0]))
            lyrics = re.findall(r'<div class="dn" id="content_h">(.*)</div>', lyric)[0]
            songs.loc[index, 'lyrics'] = lyrics                      
        except:
            still_missing[index] = (artist, track)
            print (len(still_missing))
            pass
        browser.close()

In [410]:
get_rejects(missing_songs)

In [103]:
lyrics_series = songs.lyrics

In [137]:
##removing nulls

songs_lyrics = songs[songs['lyrics'].notnull()]

In [9]:
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from collections import Counter
import time

stemmer = PorterStemmer()

In [7]:
#cleaning function

def clean(lyrics):
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    lyrics = re.sub(r'\(.*?\)', '', lyrics)
    return lyrics
    

In [12]:
songs_lyrics = songs_lyrics[~songs_lyrics['lyrics'].isnull()]

In [13]:
#running cleaning function

songs_lyrics['clean_lyrics'] = songs_lyrics['lyrics'].apply(clean)
songs_lyrics['lyrics'] = songs_lyrics['clean_lyrics']
del songs_lyrics['clean_lyrics']

In [14]:
#tokenizing strings

songs_lyrics['tokens'] = songs.lyrics.apply(lambda x: word_tokenize(str(x)))

In [15]:
progress = 0
#start = time.time()

def stem(x):
    end = time.time()
    dirty = word_tokenize(x)
    tokens = []
    for word in dirty:
        if word.strip('.') == '':
            pass
        elif re.search(r'\d{1,}', word):
            pass
        else:
            tokens.append(word.strip('.'))
    global start
    global progress
    tokens = pos_tag(tokens)
    progress += 1
    stems = ' '.join(stemmer.stem(key.lower()) for key, value in tokens if value != 'NNP')
    end = time.time()
    sys.stdout.write('\r {} percent, {} position, {} per second '.format(str(float(progress / len(songs_lyrics))), 
                                                         str(progress), (1 / (end - start))))   
    start = time.time()
    return stems

In [16]:
start = time.time()
songs_lyrics['stems'] = songs_lyrics['lyrics'].apply(stem)

 1.0 percent, 984 position, 18.621818892184198 per second 525 per second  

In [156]:
#removing duplicates

songs_lyrics = songs_lyrics.drop_duplicates(subset = 'stems', keep="first")

In [157]:
from sklearn.feature_extraction.text import CountVectorizer

In [149]:
songs_lyrics = pd.read_csv('lyrics_df_with_stems.csv')

In [158]:
vec = CountVectorizer(binary=True)

In [159]:
lyrics_array = vec.fit_transform(songs_lyrics['stems'])

In [160]:
lyrics_array.toarray().shape

(982, 6038)

In [161]:
lyrics_df = pd.DataFrame(lyrics_array.toarray())

In [162]:
lyrics_df.index = songs_lyrics.index

In [163]:
lyrics_df['year'] = songs_lyrics['year']
lyrics_df['cut'] = pd.cut(lyrics_df['year'], 12, labels = [1956, 1961, 1966, 1971, 1976, 1981, 1986, 1991, 1996, 2001, 2006, 2011])

In [488]:
word_samples

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,danceability,energy,key,loudness,speechiness,...,eighties,x,y,lyrics,iqr,iqr_year,tokens,stems,labels,length
cut,,,,,,,,,,,,,,,,,,,,,
1956,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
1963,82,82,82,82,82,82,82,82,82,82,...,82,82,82,82,82,82,82,82,82,82
1968,83,83,83,83,83,83,83,83,83,83,...,83,83,83,83,83,83,83,83,83,83
1973,132,132,132,132,132,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132
1979,71,71,71,71,71,71,71,71,71,71,...,71,71,71,71,71,71,71,71,71,71
1984,126,126,126,126,126,126,126,126,126,126,...,126,126,126,126,126,126,126,126,126,126
1989,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70
1994,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
2000,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49


In [552]:
word_samples = songs_lyrics[songs_lyrics['stems'].apply(lambda x: len(x.split(' ')) > 150)]
random.seed(42)
word_samples['sample_stems'] = word_samples.stems.apply(lambda x: random.sample(x.split(' '), 150))

/home/arvcondor/anaconda3/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [553]:
word_samples['samples'] = word_samples['sample_stems'].apply(lambda x: ' '.join(e for e in x))

/home/arvcondor/anaconda3/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [554]:
sample_array = vec.fit_transform(word_samples.samples)
sample_array = pd.DataFrame(sample_array.toarray()).reset_index()
sample_array['year'] = list(word_samples.year)
sample_array['cut'] = list(word_samples.cut)
del sample_array['index']

In [601]:
##for all songs
global_means = []
progress = 0
''' uncomment to create the dataframes of most and least common songs
least_similar_lyrics = pd.DataFrame(columns = [['song1_artist', 'song2_artist', 'song1_track', 'song2_track',
                                               'similarity', 'cut']])

most_similar_lyrics = pd.DataFrame(columns = [['song1_artist', 'song2_artist', 'song1_track', 'song2_track',
                                               'similarity', 'cut']])
'''



def get_lyric_means(lyrics_df):
    global progress
    global global_means
    
    for year in lyrics_df['cut'].unique():
        year_df = lyrics_df[lyrics_df['cut'] == year] #choose either cut or year throughout function
        del year_df['cut']
        del year_df['year']
        song_means = []
        
        for index, row in year_df.iterrows():
            song = year_df.loc[index]
           
            for indx, rowe in year_df.iterrows():
                if index == indx:
                    pass
                else:
                    songs_dot_product = np.dot(song, year_df.loc[indx])
                    song_means.append(songs_dot_product)
                    '''
                    if songs_dot_product == max(song_means):
                        most_similar_lyrics.loc[year, 'song1_artist'] = songs_lyrics.loc[index, 'artist']
                        most_similar_lyrics.loc[year, 'song2_artist'] = songs_lyrics.loc[indx, 'artist']
                        most_similar_lyrics.loc[year, 'song1_track'] = songs_lyrics.loc[index, 'track']
                        most_similar_lyrics.loc[year, 'song2_track'] = songs_lyrics.loc[indx, 'track']
                        most_similar_lyrics.loc[year, 'cut'] = year
                        most_similar_lyrics.loc[year, 'similarity'] = songs_dot_product
                    elif songs_dot_product == min(song_means):
                        least_similar_lyrics.loc[year, 'song1_artist'] = songs_lyrics.loc[index, 'artist']
                        least_similar_lyrics.loc[year, 'song2_artist'] = songs_lyrics.loc[indx, 'artist']
                        least_similar_lyrics.loc[year, 'song1_track'] = songs_lyrics.loc[index, 'track']
                        least_similar_lyrics.loc[year, 'song2_track'] = songs_lyrics.loc[indx, 'track']
                        least_similar_lyrics.loc[year, 'cut'] = year
                        least_similar_lyrics.loc[year, 'similarity'] = songs_dot_product
                    '''
            
            progress += 1
            sys.stdout.write('\r {} year, {} max, {} min, {} percent, {} position'.format(year, max(song_means), min(song_means), 
                                                                                          str(float(progress / len(songs_lyrics))), str(progress)))  
        
        global_means.append(np.nanmean(song_means))

            

In [602]:
get_lyric_means(sample_array)

 2010 year, 28 max, 4 min, 0.8839103869653768 percent, 868 positionn

In [603]:
global_means

[13.308039747064138,
 11.103896103896103,
 13.126652953276521,
 13.458362248438585,
 13.53158953722334,
 15.657269841269841,
 17.110144927536233,
 15.813519813519813,
 17.344387755102041,
 16.587122736418511,
 14.19460602855632]

In [534]:
sample_array[sample_array['cut'].isnull()]

,index,0,1,2,3,4,5,6,7,8,...,4106,4107,4108,4109,4110,4111,4112,4113,4114,cut


In [190]:
lyrics_df_cuts = pd.DataFrame(global_means)

In [191]:
lyrics_df_cuts['year'] = lyrics_df.year.unique() #change to 'cut' for cuts
lyrics_df_cuts['dotproduct'] =lyrics_df_cuts[0]
del lyrics_df_cuts[0]

In [193]:
lyrics_df_cuts.to_csv('lyrics_over_time_year.csv')

In [182]:
least_similar_lyrics['tableau_1'] = least_similar_lyrics['similarity']/2
least_similar_lyrics['tableau_2'] = -least_similar_lyrics['similarity']/2

In [183]:
least_similar_lyrics.to_csv('least_similar_lyrics.csv')

In [184]:
most_similar_lyrics['tableau_1'] = most_similar_lyrics['similarity']/2
most_similar_lyrics['tableau_2'] = -most_similar_lyrics['similarity']/2

In [185]:
most_similar_lyrics.to_csv('most_similar_lyrics.csv')

In [202]:
least_similar_lyrics

,song1_artist,song2_artist,song1_track,song2_track,similarity,cut,tableau_1,tableau_2
1961,The Troggs,Kyu Sakamoto,Wild Thing,Sukiyaki,1,1961,0.5,-0.5
1956,Bobby Boris Pickett And The Crypt-Kickers,Elvis Presley With The Jordanaires,Monster Mash,Stuck On You,0,1956,0,0
1966,Johnny Nash,The Beatles,I Can See Clearly Now,Hello Goodbye,0,1966,0,0
1971,Debby Boone,MFSB featuring The Three Degrees,You Light Up My Life,TSOP (The Sound Of Philadelphia),0,1971,0,0
1976,"Lipps, Inc.",Pink Floyd,Funkytown,Another Brick In The Wall (Part II),1,1976,0.5,-0.5
1986,New Kids On The Block,Roxette,Hangin Tough,The Look,8,1986,4,-4
1981,Belinda Carlisle,Los Lobos,Heaven Is A Place On Earth,La Bamba,0,1981,0,0
1991,Los Del Rio,The Heights,Macarena (Bayside Boys Mix),How Do You Talk To An Angel,12,1991,6,-6
1996,Alicia Keys,Enrique Iglesias,Fallin',Bailamos,10,1996,5,-5
2001,Taylor Hicks,Sean Paul,Do I Make You Proud,Get Busy,16,2001,8,-8


In [111]:
global_means

[16.202803583911219,
 13.461873410952,
 18.033779427914503,
 17.774962656811219,
 20.240050214431154,
 25.904807026347779,
 22.656974130886493,
 32.206686406832631,
 33.726695999734545,
 49.71283377820496,
 37.478169958579194,
 27.978677193218395]

In [457]:
global_means = []
progress = 0
''' uncomment to create the dataframes of most and least common songs
least_similar_lyrics = pd.DataFrame(columns = [['song1_artist', 'song2_artist', 'song1_track', 'song2_track',
                                               'similarity', 'cut']])

most_similar_lyrics = pd.DataFrame(columns = [['song1_artist', 'song2_artist', 'song1_track', 'song2_track',
                                               'similarity', 'cut']])
'''



def get_lyric_means(lyrics_df):
    global progress
    global global_means
    
    for year in lyrics_df['year'].unique():
        year_df = lyrics_df[lyrics_df['year'] == year] #choose either cut or year throughout function
        #del year_df['cut']
        del year_df['year']
        song_means = []
        
        for index, row in year_df.iterrows():
            song = year_df.loc[index]
           
            for indx, rowe in year_df.iterrows():
                if index == indx:
                    pass
                else:
                    songs_dot_product = np.linalg.norm(song - year_df.loc[indx]) #switch between np.linalg.norm to np.dot
                    song_means.append(songs_dot_product)
                    '''
                    if songs_dot_product == max(song_means):
                        most_similar_lyrics.loc[year, 'song1_artist'] = songs_lyrics.loc[index, 'artist']
                        most_similar_lyrics.loc[year, 'song2_artist'] = songs_lyrics.loc[indx, 'artist']
                        most_similar_lyrics.loc[year, 'song1_track'] = songs_lyrics.loc[index, 'track']
                        most_similar_lyrics.loc[year, 'song2_track'] = songs_lyrics.loc[indx, 'track']
                        most_similar_lyrics.loc[year, 'cut'] = year
                        most_similar_lyrics.loc[year, 'similarity'] = songs_dot_product
                    elif songs_dot_product == min(song_means):
                        least_similar_lyrics.loc[year, 'song1_artist'] = songs_lyrics.loc[index, 'artist']
                        least_similar_lyrics.loc[year, 'song2_artist'] = songs_lyrics.loc[indx, 'artist']
                        least_similar_lyrics.loc[year, 'song1_track'] = songs_lyrics.loc[index, 'track']
                        least_similar_lyrics.loc[year, 'song2_track'] = songs_lyrics.loc[indx, 'track']
                        least_similar_lyrics.loc[year, 'cut'] = year
                        least_similar_lyrics.loc[year, 'similarity'] = songs_dot_product
                    '''
            
            progress += 1
            sys.stdout.write('\r {} year, {} max, {} min, {} percent, {} position'.format(year, max(song_means), min(song_means), 
                                                                                          str(float(progress / len(songs_lyrics))), str(progress)))  
        
        global_means.append(np.nanmean(song_means))

            

In [458]:
tf_lyrics_df = pd.DataFrame(tf_lyrics_array, songs_lyrics['year']).reset_index()

In [459]:
get_lyric_means(tf_lyrics_df)

 2016 year, 1.1097281375047854 max, 0.4512376429398869 min, 1.0 percent, 982 positiont, 981 positionnn

In [453]:
tf_lyrics_df.reset_index()['year'].unique()

array([1965, 1958, 1970, 1959, 1960, 1962, 1961, 1963, 1964, 1966, 1967,
       1968, 1969, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
       1980, 1989, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1993, 1988,
       1994, 1990, 1991, 1992, 1998, 1995, 1996, 1997, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016])

In [461]:
pd.DataFrame(global_means, songs_lyrics['year'].unique()).to_csv('moretest.csv')

In [414]:
songs_lyrics[songs_lyrics['length'] == songs_lyrics['length'].max()][['artist', 'track']]

,artist,track
747,2Pac Featuring KC And JoJo,How Do U Want It/California Love


In [352]:
###GETTING JUST ONE COUNT PER WORD

lyrics_test = lyrics_df

for index, row in lyrics_test.iterrows():
    for column in lyrics_test.columns:
        if column == 'cut':
            pass
        elif lyrics_test.loc[index, column] > 0:
             lyrics_test.loc[index, column] = 1


In [386]:
global_means = []
progress = 0

get_lyric_means(lyrics_test)

 0.10365853658536585 percent, 102 position9.39836924869
 0.1991869918699187 percent, 196 position12.8016472203
 0.29573170731707316 percent, 291 position18.4201567749
 0.4502032520325203 percent, 443 position18.9790867898
 0.5284552845528455 percent, 520 position20.7399179768
 0.6697154471544715 percent, 659 position23.904181003
 0.7449186991869918 percent, 733 position28.4746390226
 0.8130081300813008 percent, 800 position32.7924016282
 0.8628048780487805 percent, 849 position48.8341836735
 0.9359756097560976 percent, 921 position39.6314553991
 1.0 percent, 984 position29.7204301075on


In [355]:
lyrics_df_one_time = pd.DataFrame(global_means)
lyrics_df_one_time['cut'] = lyrics_df.cut.unique()

In [2]:
lyrics_df = pd.read_csv('lyrics_over_time_one_hot.csv')

In [90]:
global_means = []
progress = 0
redundant = defaultdict(list)


def get_self_similarity(lyrics_df):
    global progress
    global global_means
    
    for year in lyrics_df['year'].unique():
        means = []
        year_df = lyrics_df[lyrics_df['year'] == year]
        del year_df['year']
        del year_df['cut']
        
        for index, row in year_df.iterrows():
            song = year_df.loc[index]
            song_means = []

            songs_dot_product = np.dot(song, song)
            
            song_means.append(songs_dot_product)
            
            if songs_dot_product == max(song_means):
                redundant[year] = (songs_lyrics.loc[index, 'artist'], songs_lyrics.loc[index, 'track'], 
                                   songs_dot_product)

            means.append(np.nanmean(song_means))
            
            progress += 1
            sys.stdout.write('\r {} percent, {} position'.format(str(float(progress / len(songs_lyrics))), str(progress)))  
        
        global_means.append(np.nanmean(means))

            

In [91]:
get_self_similarity(lyrics_df)

 1.0 percent, 984 positiont, 983 positionn

In [60]:
dot_product_by_year=pd.DataFrame(songs_lyrics['year'].unique())

In [92]:
global_means

[69.115384615384613,
 63.166666666666664,
 82.722222222222229,
 71.571428571428569,
 72.17647058823529,
 56.350000000000001,
 66.611111111111114,
 66.78947368421052,
 63.333333333333336,
 74.875,
 72.277777777777771,
 78.769230769230774,
 70.642857142857139,
 73.352941176470594,
 86.13636363636364,
 72.875,
 79.32352941176471,
 72.733333333333334,
 76.260869565217391,
 79.692307692307693,
 74.421052631578945,
 85.950000000000003,
 69.13333333333334,
 83.933333333333337,
 87.375,
 81.090909090909093,
 88.6875,
 82.421052631578945,
 87.478260869565219,
 80.379310344827587,
 81.692307692307693,
 117.75,
 87.290322580645167,
 88.444444444444443,
 97.333333333333329,
 85.125,
 115.11111111111111,
 114.35714285714286,
 113.55555555555556,
 134.0,
 133.66666666666666,
 101.35714285714286,
 92.13333333333334,
 135.91666666666666,
 136.875,
 172.90909090909091,
 177.27272727272728,
 134.85714285714286,
 145.77777777777777,
 120.33333333333333,
 130.69230769230768,
 124.45454545454545,
 131.8666

In [68]:
dot_product_by_year['dot_product'] = global_means

In [69]:
dot_product_by_year.to_csv('dot_product.csv')

In [37]:
lyrics_df = pd.read_csv('lyrics_df_with_stems.csv')

In [ ]:
#######################CLUSTERING############################

In [338]:
from sklearn.decomposition import TruncatedSVD

pca = TruncatedSVD(n_components=100)

In [253]:
lyrics_pca = pca.fit_transform(lyrics_array) #change between lyrics_array and tf_lyrics_array if you want tfidf scaling

In [339]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [340]:
tf = TfidfVectorizer()

tf_lyrics_array = tf.fit_transform(songs_lyrics['stems'])

tf_lyrics_array = pca.fit_transform(tf_lyrics_array)

In [380]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0).fit(lyrics_pca)

In [381]:
songs_lyrics['labels'] = kmeans.labels_

In [214]:
songs_lyrics.groupby(['labels', 'year']).count().reset_index().to_csv('lyrics_df_with_labels.csv')

In [57]:
songs_lyrics.to_csv('lyrics_df_with_labels.csv')

In [84]:
percentages = songs_lyrics.groupby(['year', 'labels']).agg({'energy': 'sum'})
percentages = percentages.groupby(level=0).apply(lambda x: 
                                                 100 * x / float(x.sum()))

In [85]:
percentages.to_csv('lyrics_percentages.csv')

In [382]:
songs_lyrics[(songs_lyrics['artist'] == '50 Cent') | (songs_lyrics['artist'] == 'Mariah Carey')][['artist','track', 'labels']]

,artist,track,labels
673,Mariah Carey,Vision Of Love,2
683,Mariah Carey,Love Takes Time,1
689,Mariah Carey,Someday,3
697,Mariah Carey,I ll Be There,0
705,Mariah Carey,Emotions,1
723,Mariah Carey,Dreamlover,2
726,Mariah Carey,Hero,1
742,Mariah Carey,Fantasy,1
745,Mariah Carey,Always Be My Baby,0
756,Mariah Carey,Honey,0


In [267]:
songs_lyrics[songs_lyrics['year'] == 2011][['artist', 'track']]

,artist,track
921,Katy Perry,Firework
922,Bruno Mars,Grenade
923,Britney Spears,Hold It Against Me
924,Wiz Khalifa,Black And Yellow
925,Lady Gaga,Born This Way
926,Katy Perry Featuring Kanye West,E.T.
927,Rihanna Featuring Britney Spears,S&M
928,Adele,Rolling In The Deep
929,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything
930,LMFAO Featuring Lauren Bennett & GoonRock,Party Rock Anthem


In [348]:
from sklearn.cluster import AffinityPropagation

In [358]:
af = AffinityPropagation(damping=.5)
af_lyrics = af.fit_predict(tf_lyrics_array)

In [321]:
songs_lyrics['labels'] = af_lyrics

In [356]:
af.cluster_centers_indices_

array([  8,  19,  34,  46,  54,  81,  95,  98, 111, 130, 148, 150, 162,
       173, 198, 212, 225, 228, 239, 243, 271, 318, 336, 361, 370, 389,
       429, 454, 460, 467, 471, 494, 511, 523, 528, 578, 584, 596, 623,
       645, 676, 707, 708, 715, 721, 725, 741, 745, 759, 777, 790, 799,
       801, 819, 820, 856, 864, 869, 897, 898, 903, 919, 925, 927, 940,
       961, 978])

In [578]:
from sklearn.cluster import AgglomerativeClustering

In [579]:
ac = AgglomerativeClustering(n_clusters=5)

In [585]:
ac.children_

array([[ 228,  336],
       [ 212,  562],
       [ 301,  946],
       ..., 
       [1957, 1958],
       [1949, 1959],
       [1960, 1961]])

In [367]:
songs_lyrics['labels'] = ac.labels_

In [379]:
songs_lyrics[songs_lyrics['labels']== 4][['track', 'artist', 'labels']]

,track,artist,labels
27,Nel Blu Dipinto Di Blu (VolarÃ©),Domenico Modugno,4
28,Little Star,The Elegants,4
56,I'm Sorry,Brenda Lee,4
85,Duke Of Earl,Gene Chandler,4
131,Rag Doll,The 4 Seasons Featuring the Sound of Frankie V...,4
252,Oh Girl,The Chi-lites,4
253,I'll Take You There,The Staple Singers,4
301,Bennie And The Jets,Elton John,4
502,Say Say Say,Paul McCartney And Michael Jackson,4
530,Sussudio,Phil Collins,4


In [387]:
songs_lyrics['length'] = songs_lyrics['stems'].apply(lambda x: len(x.split(' ')))

In [390]:
songs_lyrics.groupby('year').mean().to_csv('test_lyrics.csv')

In [576]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [595]:
Z = linkage(lyrics_pca, 'ward')

In [598]:
Z[800]

array([ 1635.        ,  1710.        ,     8.79042858,     8.        ])

In [594]:
from matplotlib import pyplot as plt
%matplotlib inline